In [13]:
import numpy as np
import pandas as pd
from spec_id import RT_spec,Gen_spec, Analyze_LH,Median_w_Error_cont,Oldest_galaxy
import matplotlib.pyplot as plt
from astropy.cosmology import Planck13, z_at_value
import astropy
from astropy import units as u
from astropy.cosmology import WMAP9 as cosmo
import os
import cPickle
import seaborn as sea
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({"xtick.direction": "in","ytick.direction": "in"})
colmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.1, as_cmap=True)

In [6]:
galDB = pd.read_pickle('../data/sgal_param_DB.pkl')

metal=np.arange(0.002,0.031,0.001)
age=np.arange(.5,6.1,.1)
tau=[0,8.0, 8.3, 8.48, 8.6, 8.7, 8.78, 8.85, 8.9, 8.95, 9.0, 9.04, 9.08, 9.11, 9.15, 9.18, 9.2, 9.23, 9.26, 9.28,
     9.3, 9.32, 9.34, 9.36, 9.38, 9.4, 9.41, 9.43, 9.45, 9.46, 9.48]
M,A=np.meshgrid(metal,age)

In [4]:
for i in galDB.index:
    P,PZ,Pt = Analyze_LH('../chidat/%s_dtau_chidata.npy' % galDB['gids'][i], galDB['hi_res_specz'][i],
                         metal,age,tau,age_conv='../data/tau_scale_delayed.dat')
    np.save('../chidat/%s_dtau_tZ_pos_ff.npy' % galDB['gids'][i],P)
    np.save('../chidat/%s_dtau_Z_pos_ff.npy' % galDB['gids'][i],[metal,PZ])
    np.save('../chidat/%s_dtau_t_pos_ff.npy' % galDB['gids'][i],[age,Pt])

In [31]:
"""DATA"""
lzDB = galDB.query('hi_res_specz < 1.16')
mzDB = galDB.query('1.16 < hi_res_specz < 1.3')
mzDB = mzDB.query('ids != 14713 and ids != 38126')
hzDB = galDB.query('1.3 < hi_res_specz < 1.45')
uhzDB = galDB.query('1.45 < hi_res_specz')

Pl,Pm,Ph,Puh = np.ones([4,len(age)])

for i in lzDB.index:
    t,Pt = np.load('../chidat/%s_79-114_t_pos_ff.npy' % lzDB['gids'][i])
    Pl = Pl * Pt
    
for i in mzDB.index:
    t,Pt = np.load('../chidat/%s_79-114_t_pos_ff.npy' % mzDB['gids'][i])
    Pm = Pm * Pt
    
for i in hzDB.index:
    t,Pt = np.load('../chidat/%s_79-114_t_pos_ff.npy' % hzDB['gids'][i])
    Ph = Ph * Pt
    
for i in uhzDB.index:
    t,Pt = np.load('../chidat/%s_79-114_t_pos_ff.npy' % uhzDB['gids'][i])
    Puh = Puh * Pt
    
Pl /= np.trapz(Pl,age)
Pm /= np.trapz(Pm,age)
Ph /= np.trapz(Ph,age)
Puh /= np.trapz(Puh,age)

lmed,ller,lher=Median_w_Error_cont(Pl,age)
mmed,mler,mher=Median_w_Error_cont(Pm,age)
hmed,hler,hher=Median_w_Error_cont(Ph,age)
uhmed,uhler,uhher=Median_w_Error_cont(Puh,age)


In [32]:
AGE = Oldest_galaxy(np.median(lzDB['hi_res_specz'])) - lmed + Oldest_galaxy(np.median(mzDB['hi_res_specz'])) - mmed +\
      Oldest_galaxy(np.median(hzDB['hi_res_specz'])) - hmed + Oldest_galaxy(np.median(uhzDB['hi_res_specz'])) - uhmed 

print AGE/4.

1.58416158482


In [34]:
print  Oldest_galaxy(0) * u.Gyr -Planck13.lookback_time(3.9)

1.58855753449 Gyr
